In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from privacypacking.utils.utils import LOGS_PATH
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve
import yaml

In [ ]:
print(yaml.dump(LaplaceCurve(1/5.0).epsilons))
print()
print(yaml.dump(LaplaceCurve(1/4.0).epsilons))

In [ ]:
# logs = LOGS_PATH.joinpath("ray/run_and_report_2021-10-29_17-22-50")

# df = load_ray_experiment(logs)

df = load_latest_ray_experiment()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
px.line(df.sort_values('n_blocks'), x="n_blocks", y="n_allocated_tasks", color="scheduler", title="Multiblock DPF killer", facet_col="block_selecting_policy")

In [ ]:
# Zeta
alpha = 0.1
n_blocks = 10
density = np.array([1 / (i + 1) ** (alpha) for i in range(n_blocks)])
density /= sum(density)

px.line(density)